In [2]:
{-# LANGUAGE InstanceSigs #-}
{-# LANGUAGE TypeFamilies #-}

import Control.Applicative
import GHC.Exts (IsList(..))

data List a = Nil | Cons a (List a) deriving (Eq, Show)

instance IsList (List a) where
    type Item (List a) = a
    fromList = foldr Cons Nil
    toList   = foldr (:) []

In [41]:
instance Functor List where
    fmap :: (a -> b) -> List a -> List b
    fmap f list = case list of
        Nil      -> Nil
        Cons h t -> Cons (f h) (fmap f t)

instance Foldable List where
    foldr :: (a -> b -> b) -> b -> List a -> b
    foldr f z ls = case ls of
        Nil -> z
        Cons h t -> f h (foldr f z t)

In [17]:
append :: List a -> List a -> List a
append a b = case a of
    Nil -> b
    Cons h t -> Cons h (append t b)

catenate :: List (List a) -> List a
catenate lists = case lists of
    Nil -> Nil
    Cons h t -> append h (catenate t)
    

instance Applicative List where
    pure :: a -> List a
    pure a = Cons a Nil
    
    (<*>) :: List (a -> b) -> List a -> List b
    fs <*> as = case fs of
        Nil        -> Nil
        Cons f fs' -> append (f <$> as) (fs' <*> as)

In [25]:
instance Alternative List where
    empty :: List a
    empty = Nil
    
    (<|>) :: List a -> List a -> List a
    (<|>) = append

In [31]:
instance Monad List where
    (>>=) :: List a -> (a -> List b) -> List b
    as >>= f = catenate (f <$> as)

In [56]:
import Control.Monad (guard)

evenHalves ls = do
    a <- ls
    guard (a `mod` 2 == 0)
    let b = a `div` 2
    return b

evenHalves (fromList [1..10]) :: List Int

Cons 1 (Cons 2 (Cons 3 (Cons 4 (Cons 5 Nil))))